# QA bot baseline
## Pretrained Model Load(SKT-KoGPT2)

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>',
                                                    eos_token='</s>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')
#tokenizer.tokenize("</s> 안녕하세요. 한국어 GPT-2 입니다.😤:)l^o </s>")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
tokenizer.tokenize("</s> 봄 에 입을 옷 종류를 추천해주세요. </s>")

['</s>', '▁봄', '▁에', '▁입을', '▁옷', '▁종류를', '▁추천', '해주', '세', '요.', '▁', '</s>']

In [ ]:
tokenizer.tokenize("</s> 봄에 입을 옷 종류를 추천해주세요. </s>")

['</s>', '▁봄에', '▁입을', '▁옷', '▁종류를', '▁추천', '해주', '세', '요.', '▁', '</s>']

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

# Data Preprocessing

In [ ]:
# 구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json

def load_data(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    pairs = []
    for entry in data["data_info"]:
        question = entry["question"]
        answer = entry["answer"]["contents"]
        pairs.append((question, answer))

    return pairs

In [ ]:
filepath_test = '/content/drive/MyDrive/Project2/Data/SFTlabel.json'
pairs_test = load_data(filepath_test)
print(pairs_test[3])

('도자기를 고르게 돌리기 위해 사용하는 도구의 이름은?', "도자기를 고르게 돌리기 위해 사용하는 도구는 '도마'입니다. 도마는 도자기를 고르게 회전시켜주는 원형의 도구로서 주로 도자기 공방이나 도자기 작업 공간에서 사용됩니다.")


In [ ]:
filepath_validation = '/content/drive/MyDrive/Project2/Data/SFTlabel_Validation.json'
pairs_validation = load_data(filepath_validation)
print(pairs_validation[3])

('유리공예 작업할 때 필요한 도구와 재료가 뭐야?', '유리공예 작업을 위해 필요한 도구 및 재료는 다음과 같습니다:\n\n도구\n1. 유리 절단 칼: 유리를 원하는 크기로 절단하기 위해 사용됩니다.\n2. 선과: 직선을 그리기 위해 사용됩니다.\n3. 절단 접착제: 접합을 위해 사용되며, 강력한 접착제로 유리 조각이 붙는 데 도움을 줍니다.\n4. 모래 지속기: 유리의 끝을 다듬기 위해 사용됩니다.\n5. 소화기: 작업 도중 작은 불꽃이 발생할 경우 안전을 위해 사용됩니다.\n6. 연마 도구: 유리의 표면을 연마하여 다듬을 수 있습니다.\n7. 벤치 그라인더: 유리를 모양을 다듬는 데 사용되며, 모양을 좀 더 정확하게 조정할 수 있습니다.\n8. 페인트 브러시: 디자인과 색상을 유리에 그릴 때 사용됩니다.\n\n재료\n1. 유리 조각: 다양한 크기, 모양, 색상의 유리 조각을 사용할 수 있습니다.\n2. 테두리: 유리 작품을 보호하거나 견고하게 만들기 위해 사용됩니다.\n3. 줄무늬: 작품을 걸거나 장식할 때 사용되며, 다양한 색상과 재질을 사용할 수 있습니다.\n4. 트림: 테두리를 꾸미거나 작품의 외관을 강조하기 위해 사용됩니다.\n5. 유리그릇: 작품에 사용될 수 있는 꽃, 모래, 컬러드 샌드 등의 재료를 담을 수 있습니다.\n6. 폴리싱 제품: 유리의 표면을 광택 내기 위해 사용됩니다.\n7. 에나멜 그릇: 유리에 그림을 그리고 색칠하기 위해 사용됩니다.\n8. 용제: 유리 조각을 부착하거나 다른 재료를 붙이기 위해 사용됩니다.\n\n다양한 도구와 재료를 사용하여 유리공예 작업을 즐기고 다양한 작품을 만들 수 있습니다!')


In [ ]:
# for param in model.transformer.parameters():
#     param.requires_grad = False

# for param in model.lm_head.parameters():
#     param.requires_grad = True

In [ ]:
import torch
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, qa_pairs, tokenizer, max_length=256):
        self.qa_pairs = qa_pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.qa_pairs)

    def __getitem__(self, idx):
        question, answer = self.qa_pairs[idx]
        input_text = f"</s> 질문: {question} 답변: {answer} </s>"

        encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        # labels를 input_ids의 복사본으로 생성
        labels = input_ids.clone()

        # 패딩 토큰에 대한 손실을 무시하기 위해 패딩 토큰 인덱스를 -100으로 설정
        labels[input_ids == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }



In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataset = QADataset(pairs_test, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = QADataset(pairs_validation, tokenizer)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
# DataLoader에서 배치 샘플 확인

print(len(train_dataloader))
print(len(train_dataloader.dataset))

for batch in train_dataloader:
    print(batch['input_ids'].shape)  # (batch_size, max_length)
    print(batch['attention_mask'].shape)  # (batch_size, max_length)
    print(batch['labels'].shape)  # (batch_size, max_length)
    break

print(len(val_dataloader))
print(len(val_dataloader.dataset))

for batch in val_dataloader:
    print(batch['input_ids'].shape)  # (batch_size, max_length)
    print(batch['attention_mask'].shape)  # (batch_size, max_length)
    print(batch['labels'].shape)  # (batch_size, max_length)
    break

662
10580
torch.Size([16, 256])
torch.Size([16, 256])
torch.Size([16, 256])
83
1322
torch.Size([16, 256])
torch.Size([16, 256])
torch.Size([16, 256])


## Model Fine-tuning

In [ ]:
def save_model(model, tokenizer, epoch):
  save_directory = '/content/drive/MyDrive/Project2/fine_tuned2_kogpt2_epoch' + str(epoch+1)

  model.save_pretrained(save_directory)
  tokenizer.save_pretrained(save_directory)

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

num_epochs = 30


# Dataset과 DataLoader 정의
# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

# 옵티마이저와 스케줄러 정의
optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=total_steps)

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    model.eval()  # 검증 모드
    val_loss = 0
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, labels=labels)
            val_loss += outputs.loss.item()

    val_loss /= len(val_dataloader)  # 평균 검증 손실 계산
    print(f"Epoch: {epoch}, Loss: {loss.item()}, Validation Loss: {val_loss}")

    if (epoch + 1) % 5 == 0:
        save_model(model, tokenizer, epoch)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 0, Loss: 2.5498781204223633, Validation Loss: 2.6303213619324097
Epoch: 1, Loss: 2.5196614265441895, Validation Loss: 2.554392087890441
Epoch: 2, Loss: 2.2004594802856445, Validation Loss: 2.5338284710803665
Epoch: 3, Loss: 2.026648759841919, Validation Loss: 2.5389505380607513
Epoch: 4, Loss: 1.7974423170089722, Validation Loss: 2.549360525177186
Epoch: 5, Loss: 2.128162384033203, Validation Loss: 2.56954034839768
Epoch: 6, Loss: 1.9983564615249634, Validation Loss: 2.598156104604882
Epoch: 7, Loss: 1.7069597244262695, Validation Loss: 2.6202958664262153
Epoch: 8, Loss: 1.5984896421432495, Validation Loss: 2.6580891896443193
Epoch: 9, Loss: 1.41572105884552, Validation Loss: 2.6896425902125345
Epoch: 10, Loss: 1.3024977445602417, Validation Loss: 2.726694095565612
Epoch: 11, Loss: 1.3147656917572021, Validation Loss: 2.758708973965013
Epoch: 12, Loss: 1.1436717510223389, Validation Loss: 2.7997053387653397
Epoch: 13, Loss: 0.9771098494529724, Validation Loss: 2.822834138410637


KeyboardInterrupt: 

## Finish

In [ ]:
# 코랩 세션을 종료시켜주세요
from google.colab import runtime
runtime.unassign()